### Check files dimension in the database

In [ ]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path
import os

st = study('Olfacto')
path_data = path.join(st.path,'database/Encoding_By_Odor/try_PIRJ/') #Retrieval_EpiPerf_LowHigh
files = os.listdir(path_data) #Retrieval_By_Odor

for fi in files:
    if fi.endswith('ok.npz') and fi.startswith('PIRJ'):
        loadname = path_data+fi
        mat = np.load(loadname, allow_pickle=True)
        x = np.load(loadname)['x']
        print (fi, x.shape,mat.files, mat['channel'][:10])

## Remove bad electrodes after artefact detection
### TF with artefacts or 50Hz

In [ ]:
import numpy as np
from brainpipe.system import study
from os import path

In [ ]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/R_pre_stim_1s_By_Odor/')
#conds = ['low', 'high']
phases = ['odor']#'rest'
rej_elec = {
    'CHAF' : [98,102,103,104],
    'VACJ' : [0,66,67,68,122,128], 
    'SEMC' : [36],
    'PIRJ' : [10,14,15,25,26,41,47,48,53,54,55,57,58,63,64,71,72,79,80,81,82,83,],
    'LEFC' : [47,48,49,50,140,141,],
    'MICP' : [4,9,13,71,],
    'FERJ': [],
            }
files = [k for k in st.search('_bipo.npz', folder=('database/R_pre_stim_1s_By_Odor/'))]
for fi in files:
    mat = np.load(pathfiles+fi)
    x, sf, label, channel, xyz = mat['x'], mat['sf'], mat['label'], mat['channel'], mat['xyz']
    print (fi, x.shape, label.shape, channel.shape, xyz.shape)

    #Remove all bad electrodes from the database
    new_x = np.delete(x,rej_elec[fi[:4]], axis=0)
    new_label = np.delete(label,rej_elec[fi[:4]], axis=0)
    new_channel = np.delete(channel,rej_elec[fi[:4]], axis=0)
    new_xyz = np.delete(xyz,rej_elec[fi[:4]], axis=0)
    print (fi, len(rej_elec[fi[:4]]),new_x.shape, new_label.shape, new_channel.shape, new_xyz.shape)

    #Save all new information
    mat_new = {'x':new_x, 'label':new_label, 'channel':new_channel, 'xyz':new_xyz, 'sf':sf}
    file_source = fi.replace('bipo.npz','bipo2.npz')
    np.savez(pathfiles+file_source,**mat_new)

In [ ]:
"""Remove aHC electrodes in PIRJ (all but O6/O7)"""
from utils import odor_su_score
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/Encoding_By_Odor/try_PIRJ/')
files = path.join(pathfiles, 'PIRJ_odor_{}_bipo.npz')

ref = np.load(files.format('6'),allow_pickle=True)['channel']

d_sel = {}
for od in odor_su_score['PIRJ']:
    mat = np.load(files.format(od),allow_pickle=True)
    idx_ = [i for i,ch in enumerate(mat['channel']) if ch in ref]
    
    for f in mat.files:
        d_sel[f] = mat[f][idx_] if f != 'sf' else mat[f]
    np.savez(files.format(od).replace('.npz','_elecs_ok.npz'),**d_sel)

### Remove elecs with WM

In [ ]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/Encoding_By_Odor/try_PIRJ/')

files = [k for k in st.search('elecs_ok.npz', folder=('database/Encoding_By_Odor/try_PIRJ/'))]
for fi in files:
    mat = np.load(pathfiles+fi)
    print(mat.files)
    data, label, channel, xyz,sf= mat['x'], mat['label'], mat['channel'], mat['xyz'],mat['sf']
    print(fi, 'data shape: ', data.shape, label.shape, channel.shape, xyz.shape)

    #create a list of all electrodes to remove
    elecs = label.shape[0]
    elecs_rej = []
    for elec in range(elecs):
        elec_rej_wm = np.where(label[elec].find('WM') != -1,int(elec),None)
        elec_rej_lcr = np.where(label[elec].find('LCR') != -1,int(elec),None)
        elec_rej_skull = np.where(label[elec].find('skull') != -1,int(elec),None)
        elec_rej_out = np.where(label[elec].find('out') != -1,int(elec),None)
        if elec_rej_wm != None:
            elecs_rej = np.append(elecs_rej,elec_rej_wm)
        if elec_rej_lcr != None and elec_rej_lcr != elec_rej_wm:
            elecs_rej = np.append(elecs_rej,elec_rej_lcr)
        if elec_rej_skull != None and elec_rej_skull != elec_rej_wm and elec_rej_skull != elec_rej_lcr:
            elecs_rej = np.append(elecs_rej,elec_rej_skull)
        if elec_rej_out != None and elec_rej_out != elec_rej_skull and elec_rej_out!= elec_rej_wm and elec_rej_out != elec_rej_lcr:
            elecs_rej = np.append(elecs_rej, elec_rej_out)

    #Remove all bad electrodes from the database
    new_data, new_label = np.delete(data,elecs_rej, axis=0), np.delete(label,elecs_rej, axis=0)
    new_channel, new_xyz = np.delete(channel,elecs_rej, axis=0), np.delete(xyz,elecs_rej, axis=0)
    print (fi,'new data',new_data.shape, new_label.shape,new_channel.shape, new_xyz.shape)

    #Save all new information
    mat_new = {'x':new_data, 'label':new_label, 'channel':new_channel, 'xyz':new_xyz, 'sf':sf}
    filename2 = fi.replace('elecs_ok.npz','all_noWM.npz')
    print('CHECK', mat_new['x'].shape)
    np.savez(pathfiles+filename2,**mat_new)
    del elecs, elecs_rej, new_data, new_label, new_channel, new_xyz

### Add AAL & BA labels to subjects' matrices
### Add also the Mai labels (Jane-AL) post fMRI

In [ ]:
#CHECK NUMBER OF ELECTRODES WITHOUT ARTEFACTS 
PATHS = [path.join(st.path,'database/R_pre_stim_1s/'),
        path.join(st.path,'database/R_resting_3s/'),
        path.join(st.path,'database/Retrieval_EpiPerf_LowHigh/')]
subjects = ['CHAF','SEMC','PIRJ','FERJ','LEFC','VACJ']
for su in subjects:
    for p in PATHS:
        filename = p+'_labels/Elecs_{}_labels_aal.csv'.format(su)
        df = pd.read_csv(filename)
        print(p, su, df.shape)

In [ ]:
#############SANS LABELS DE MAI#######################################
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('Olfacto')
path_data = path.join(st.path,'database/Encoding_By_Odor/try_PIRJ/')
path_labels = path.join(st.path,'database/Encoding_By_Odor/_labels/') #create with visbrain in 4_Visbrain new scripts

files = [k for k in st.search('bipo_all_noWM.npz', folder=('database/Encoding_By_Odor/try_PIRJ/'))]
for f in files:
    mat = np.load(path_data+f, allow_pickle=True)
    kwargs = {}
    kwargs['x'], kwargs['channels'], kwargs['xyz'] = mat['x'], mat['channel'],mat['xyz']
    kwargs['labels'], kwargs['sf']=mat['label'],mat['sf']
    # Load the AAL labels
    aal = 'Elecs_'+f[:4]+'_labels_aal.csv'
    df = pd.read_csv(path_labels+aal, sep=',')
    kwargs['aal'] = df['aal'].values
    # Load the BA labels
    BA = 'Elecs_'+f[:4]+'_labels_brodmann.csv'
    df = pd.read_csv(path_labels+BA, sep=',')
    kwargs['BA'] = df['brodmann'].values
    # Update the datafile
    filename = f.replace('noWM.npz','noWM_phys.npz')
    np.savez(path_data+filename, **kwargs)
    mat2 = np.load(path_data+filename, allow_pickle=True)
    print(f[:4],mat2['x'].shape, mat2['xyz'].shape, mat2['aal'].shape, mat2['BA'].shape)
    

In [ ]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('Olfacto')
path_data = path.join(st.path,'database/Encoding_By_Odor/try_PIRJ/')
path_elecs = path.join(st.path,'database/Encoding_By_Odor/')
path_labels = path.join(path_elecs,'All_elecs_infos_npz/') #create with visbrain in 4_Visbrain new scripts
path_atlas = path.join(path_elecs, '_labels/')
files = [k for k in st.search('bipo_all_noWM_phys.npz', 
                              folder=('database/Encoding_By_Odor/try_PIRJ'))]

for fi in files:
    mat = np.load(path_data+fi)
    print(mat.files)
    kwargs = {}
    kwargs['x'], kwargs['channels'], kwargs['xyz'] = mat['x'], mat['channels'],mat['xyz']
    kwargs['labels'], kwargs['sf']=mat['labels'],mat['sf']
    # Load the AAL labels
    aal = 'Elecs_'+fi[:4]+'_labels_aal.csv'
    df = pd.read_csv(path_atlas+aal, sep=',')
    kwargs['aal'] = df['aal'].values
    # Load the BA labels
    BA = 'Elecs_'+fi[:4]+'_labels_brodmann.csv'
    df = pd.read_csv(path_atlas+BA, sep=',')
    kwargs['BA'] = df['brodmann'].values
    #Load the new_labels from Mai atlas
    Mai_file = '0_all_subjects_info_elecs_mai_labels.csv'
    df2 = pd.read_csv(path_labels+Mai_file, sep=',')
    Mai_data = df2['Mai_RL'].loc[df2['su_names'] == fi[:4]].values
    kwargs['Mai_RL'] = Mai_data
    #Add  the new labels Mai RL
    kwargs['Mai'] = Mai_data+'_'+['R' if x > 0 else 'L' for x in df2['x'].loc[df2['su_names'] == fi[:4]].values]
    # Update the datafile
    filename2 = fi.replace('bipo_all_noWM_phys.npz','bipo_all_noWM_phys.npz')
    np.savez(path_data+filename2, **kwargs)
    mat2 = np.load(path_data+filename2, allow_pickle=True)
    print(mat2.files, mat2['Mai'][:5],mat2['Mai_RL'][:5])
    print(fi[:4],mat2['x'].shape, mat2['xyz'].shape, mat2['aal'].shape, mat2['BA'].shape,
          mat2['Mai'].shape, mat2['Mai_RL'].shape)

### Select only regions present in at least 2 patients 
#### Remove all not memory related regions (motor, precentral, SMA)

In [ ]:
st = study('Olfacto')
pathfiles = path.join(st.path, 'database/Encoding_EpiPerf_LowHigh/')
subjects = ['CHAF','VACJ','SEMC','LEFC','MICP','PIRJ','FERJ']
conds = ['bad','good']
to_keep = ['Amygdala (R)', 'Cingulum Ant (R)','Cingulum Mid (R)', 'Frontal Inf Orb (R)', 
           'Frontal Inf Tri (R)', 'Frontal Mid (R)', 'Frontal Mid Orb (R)', 'Frontal Sup (R)',
           'Frontal Sup Medial (R)', 'Frontal Sup Orb (R)', 'Hippocampus (R)', 'Insula (R)',
           'ParaHippocampal (R)','Temporal Inf (R)', 'Temporal Mid (R)', 'Temporal Pole Mid (R)',
           'Temporal Pole Sup (R)', 'Temporal Sup (R)','Amygdala (L)', 'Cingulum Ant (L)', 
           'Cingulum Mid (L)', 'Frontal Inf Orb (L)', 'Frontal Inf Tri (L)','Frontal Mid (L)', 
           'Frontal Mid Orb (L)', 'Frontal Sup (L)','Frontal Sup Medial (L)', 'Frontal Sup Orb (L)',
           'Hippocampus (L)', 'Insula (L)','ParaHippocampal (L)','Temporal Inf (L)', 
           'Temporal Mid (L)','Temporal Pole Mid (L)','Temporal Pole Sup (L)', 'Temporal Sup (L)']

for su in subjects:
    for cond in conds:
        mat = np.load(pathfiles+su+'_odor_'+cond+'_bipo_sel_phys.npz')
        idx = np.where([mat['phys'][i] in to_keep for i in range(len(mat['phys']))])
        kwargs = {}
        kwargs['labels'], kwargs['x'], kwargs['phys'] = mat['labels'][idx], mat['x'][idx], mat['phys'][idx]
        kwargs['xyz'], kwargs['channels'], kwargs['sf'] = mat['xyz'][idx], mat['channels'][idx], mat['sf']
        np.savez(pathfiles+su+'_odor_'+cond+'_bipo_sel_phys2.npz',**kwargs)
        del mat

### take same electrodes at retrieval than encoding

In [ ]:
from brainpipe.system import study
import numpy as np
from os.path import join

st= study('Olfacto')
PATH_E = join(st.path, 'database/Encoding_By_Odor/')
PATH_R = join(st.path, 'database/Encoding_No_Odor/')

filesE = st.search('all_noWM_physFT.npz',folder ='database/Encoding_By_Odor/' )

for fi in filesE:
    matE = np.load(PATH_E+fi,allow_pickle=True)
    matR = np.load(PATH_R+fi.replace('_all_noWM_physFT',''),allow_pickle=True)
    channelsE = matE['channels']
    channelsR = matR['channel']
    idx_sel_R = [i for i,chan in enumerate(channelsR) if chan in channelsE]
    idx_sel_E = [i for i,chan in enumerate(channelsE) if chan in channelsR]
    
    dict_data = {}
    for file in matE.files:
        dict_data[file] = matE[file][idx_sel_E] if file != 'sf' else matE[file]
    dict_data['x'] = matR['x'][idx_sel_R,...]
    np.savez(PATH_R+fi,**dict_data)
    dict_data['x'] = matE['x'][idx_sel_E,...]
    np.savez(PATH_E+fi.replace('physFT','physFT_reinstatement'),**dict_data)

### Concatenate Low, Mid, High conditions

In [ ]:
import numpy as np
from brainpipe.system import study
from utils import odor_groups_wgth as dict_ #odor_groups_wgth, odor_groups_3wgth
from os.path import join, exists
from os import makedirs
from itertools import product

st = study('Olfacto')
PATH = join(st.path, 'database/Encoding_By_Odor/')
od_name = join(PATH, '{}_odor_{}_bipo_all_noWM_physFT.npz')
PATH_COND = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/2conds/')
save_name = join(PATH_COND, '{}_odor_{}_E.npz')
subjects = ['PIRJ']
    
for su in subjects:
    for cond in dict_[su]:
        print('>>> processing:', su, cond)
        glob_data = np.array([])
        for od in dict_[su][cond]:
            mat = np.load(od_name.format(su,od),allow_pickle=True)
            glob_data = np.concatenate((glob_data, mat['x']),
                               axis=-1) if np.size(glob_data) else mat['x']
            print(su, od, cond,glob_data.shape)
        dict_pow = {}
        for file in mat.files:
            dict_pow[file] = mat[file]
        dict_pow['x'] = glob_data
        np.savez(save_name.format(su,cond),**dict_pow)

In [2]:
def sep_early_late(data):
    thr = int(data.shape[-1]/2)
    if thr < 1:
        early = data
        late = np.array([])
    else:
        early = data[...,:thr]
        late = data[...,thr:]
    return early, late

In [4]:
"""
Separate each odor in Early and Late parts 
"""

import numpy as np
from brainpipe.system import study
from utils import su_list_od #odor_groups_wgth, odor_groups_3wgth
from os.path import join, exists
from os import makedirs
from itertools import product

st = study('Olfacto')
PATH = join(st.path, 'database/Encoding_By_Odor/')
od_name = join(PATH, '{}_odor_{}_bipo_all_noWM_physFT.npz')
PATH_COND = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/')
save_name = join(PATH_COND, '{}_odor_early_late.npz')
    
for su in su_list_od:
    early_glob, late_glob = np.array([]), np.array([])
    for od in su_list_od[su]:
        print('>>> processing:', su, od)
        mat = np.load(od_name.format(su,od),allow_pickle=True)
        data = mat['x']
        early,late = sep_early_late(data)
        early_glob = np.concatenate((early_glob, early),
                           axis=-1) if np.size(early_glob) else early
        if np.size(late):
            late_glob = np.concatenate((late_glob, late),
                           axis=-1) if np.size(late_glob) else late
        print(su, od,early_glob.shape,late_glob.shape)
        
        dict_pow = {}
        for file in mat.files:
            dict_pow[file] = mat[file]
        dict_pow['x_E'] = early_glob
        dict_pow['x_L'] = late_glob
        np.savez(save_name.format(su),**dict_pow)

-> Olfacto loaded
>>> processing: CHAF 1
CHAF 1 (61, 3584, 1) (0,)
>>> processing: CHAF 2
CHAF 2 (61, 3584, 2) (61, 3584, 2)
>>> processing: CHAF 4
CHAF 4 (61, 3584, 3) (61, 3584, 2)
>>> processing: CHAF 5
CHAF 5 (61, 3584, 4) (61, 3584, 2)
>>> processing: CHAF 3
CHAF 3 (61, 3584, 5) (61, 3584, 4)
>>> processing: CHAF 8
CHAF 8 (61, 3584, 6) (61, 3584, 6)
>>> processing: CHAF 7
CHAF 7 (61, 3584, 8) (61, 3584, 8)
>>> processing: CHAF 9
CHAF 9 (61, 3584, 10) (61, 3584, 11)
>>> processing: VACJ 11
VACJ 11 (39, 3584, 1) (39, 3584, 2)
>>> processing: VACJ 14
VACJ 14 (39, 3584, 2) (39, 3584, 4)
>>> processing: VACJ 12
VACJ 12 (39, 3584, 3) (39, 3584, 5)
>>> processing: VACJ 10
VACJ 10 (39, 3584, 4) (39, 3584, 7)
>>> processing: VACJ 15
VACJ 15 (39, 3584, 5) (39, 3584, 8)
>>> processing: VACJ 17
VACJ 17 (39, 3584, 6) (39, 3584, 9)
>>> processing: VACJ 16
VACJ 16 (39, 3584, 7) (39, 3584, 11)
>>> processing: VACJ 13
VACJ 13 (39, 3584, 9) (39, 3584, 13)
>>> processing: SEMC 7
SEMC 7 (53, 3584, 2)

In [8]:
"""
Separate each odor in Early and Late parts + MEMORY COND
"""

import numpy as np
from brainpipe.system import study
from utils import su_list_od, odor_groups_wgth #odor_groups_wgth, odor_groups_3wgth
from os.path import join, exists
from os import makedirs
from itertools import product

st = study('Olfacto')
PATH = join(st.path, 'database/Encoding_By_Odor/')
od_name = join(PATH, '{}_odor_{}_bipo_all_noWM_physFT.npz')
PATH_COND = join(st.path, 'database/Encoding_By_Cond_v=1_elecs=all/')
save_name = join(PATH_COND, '{}_odor_cond={}_early_late.npz')
    
for su in odor_groups_wgth:
    for cond in odor_groups_wgth[su]:
        early_glob, late_glob = np.array([]), np.array([])
        for od in odor_groups_wgth[su][cond]:
            print('>>> processing:', su, od,cond)
            mat = np.load(od_name.format(su,od),allow_pickle=True)
            data = mat['x']
            early,late = sep_early_late(data)
            early_glob = np.concatenate((early_glob, early),
                               axis=-1) if np.size(early_glob) else early
            if np.size(late):
                late_glob = np.concatenate((late_glob, late),
                               axis=-1) if np.size(late_glob) else late
            print(su, od,early_glob.shape,late_glob.shape)

            dict_pow = {}
            for file in mat.files:
                dict_pow[file] = mat[file]
            dict_pow['x_E'] = early_glob
            dict_pow['x_L'] = late_glob
            np.savez(save_name.format(su,cond),**dict_pow)

-> Olfacto loaded
>>> processing: CHAF 1 low
CHAF 1 (61, 3584, 1) (0,)
>>> processing: CHAF 2 low
CHAF 2 (61, 3584, 2) (61, 3584, 2)
>>> processing: CHAF 4 low
CHAF 4 (61, 3584, 3) (61, 3584, 2)
>>> processing: CHAF 5 low
CHAF 5 (61, 3584, 4) (61, 3584, 2)
>>> processing: CHAF 3 high
CHAF 3 (61, 3584, 1) (61, 3584, 2)
>>> processing: CHAF 8 high
CHAF 8 (61, 3584, 2) (61, 3584, 4)
>>> processing: CHAF 7 high
CHAF 7 (61, 3584, 4) (61, 3584, 6)
>>> processing: CHAF 9 high
CHAF 9 (61, 3584, 6) (61, 3584, 9)
>>> processing: VACJ 11 low
VACJ 11 (39, 3584, 1) (39, 3584, 2)
>>> processing: VACJ 14 low
VACJ 14 (39, 3584, 2) (39, 3584, 4)
>>> processing: VACJ 12 low
VACJ 12 (39, 3584, 3) (39, 3584, 5)
>>> processing: VACJ 10 low
VACJ 10 (39, 3584, 4) (39, 3584, 7)
>>> processing: VACJ 15 high
VACJ 15 (39, 3584, 1) (39, 3584, 1)
>>> processing: VACJ 17 high
VACJ 17 (39, 3584, 2) (39, 3584, 2)
>>> processing: VACJ 16 high
VACJ 16 (39, 3584, 3) (39, 3584, 4)
>>> processing: VACJ 13 high
VACJ 13 (39

### Take same electrodes and labels Odor and No Odor

In [23]:
from brainpipe.system import study
import numpy as np
from os.path import join
from utils import su_list_od

st= study('Olfacto')
filename = join(st.path,
             'database/Encoding_By_Odor/{}_odor_{}_bipo_all_noWM_physFT.npz')
file_no_od = join(st.path,
             'database/Encoding_No_Odor/{}_odor_no_odor_bipo_sel_phys.npz')

files_to_match = []
for su in su_list_od:
    for od in su_list_od[su][:1]:
        files_to_match.append(filename.format(su,od))

for fi in files_to_match:
    match = np.load(fi,allow_pickle=True)
    su = fi.split('/')[-1][:4]
    mat_no_odor = np.load(file_no_od.format(su),allow_pickle=True)
    channels_m = match['channels']
    channels_no_od = mat_no_odor['channels']
    idx_sel = [i for i,chan in enumerate(channels_no_od) if chan in channels_m]
    
    dict_data = {}
    for file in match.files:
        dict_data[file] = match[file] if file != 'sf' else match[file]
    dict_data['x'] = mat_no_odor['x'][idx_sel,...]
    print(su,match['Mai_RL'])
    np.savez(file_no_od.format(su).replace('phys.npz','physFT.npz'),**dict_data)

-> Olfacto loaded
CHAF ['SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG'
 'ACC' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'ACC'
 'ACC' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG'
 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'MFG' 'OFC_olf'
 'OFC_olf' 'OFC_olf' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'MFG'
 'MFG' 'MFG' 'MFG']
VACJ ['aHC' 'aHC' 'aHC' 'aHC' 'aHC' 'aHC' 'aHC' 'aHC' 'Amg' 'Amg' 'Amg' 'IFG'
 'IFG' 'IFG' 'IFG' 'IFG' 'MFG' 'MFG' 'pPirT' 'ACC' 'MFG' 'MFG' 'MFG' 'MFG'
 'MFG' 'MFG' 'ACC' 'IFG' 'IFG' 'IFG' 'IFG' 'IFG' 'IFG' 'OFC_olf' 'OFC_olf'
 'OFC_olf' 'OFC_olf' 'IFG' 'IFG']
SEMC ['aHC' 'aHC' 'aHC' 'aHC' 'aHC' 'IFG' 'IFG' 'IFG' 'IFG' 'IFG' 'IFG' 'MFG'
 'MFG' 'MFG' 'IFG' 'IFG' 'ACC' 'MFG' 'MFG' 'MFG' 'MFG' 'ACC' 'OFC_olf'
 'OFC_olf' 'OFC_olf' 'IFG' 'IFG' 'IFG' 'IFG' 'ACC' 'IFG' 'IFG' 'IFG' 'IFG'
 'IFG' 'ACC' 'MFG' 'MFG' 'MFG' 'MFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'SFG'
 'SFG' 'SFG' 'SFG' 'SFG' 'SFG' 'M